coroutine <br>

- Single thread로 stack을 기반으로 동작하는 비동기 작업 
- 상태 저장(클로저), 양방향 전송

Thread: OS에서 관리함, CPU core에서 실시간 시분할로 비동기작업을 진행한다. <br>
yield를 이용해서 main routine과 subroutine이 상호작용할 것이다. <br>

- subroutine: 메인루틴에서 호출한다. 서브루틴에서 수행함
- coroutine: 루틴 실행 중 중지 -> 다른 work로 switching -> 동시성 프로그래밍
- 코루틴은 여러 쓰레드에 비해 overhead가 감소한다.

<br>
멀티쓰레드는 복잡하고 자원이 공유되어 deadlock(교착상태)에 빠질 수 있으며 <br>
context switching(Thread 교체) 비용이 발생한다. <br>

In [10]:
# 코루틴1
# 함수형 -> 제네레이터 -> 메시지 ->코루틴

# generator based on function
def coroutine1():
    # subroutine
    print('>> coroutine started.')
    i = yield

    print(f'>> coroutine received : {i}')


# coroutine based on generator

# main routine
cr1 = coroutine1()
print(cr1, type(cr1))




<generator object coroutine1 at 0x00000272F04A0D60> <class 'generator'>


In [11]:
# subroutine이 main routine에 종속적이다.
# 메인에서 하라는 대로 했다.
next(cr1)
next(cr1)

>> coroutine started.
>> coroutine received : None


StopIteration: 

In [12]:
# send로 루틴끼리 양방향으로 상효작용 기능하다. 
cr1 = coroutine1()

# next가 yield 지점까지 서브루틴을 수행한다.
next(cr1)

cr1.send(100)
# send가 yield에서 i에 값을 할당하고 next의 기능을 수행한다.
# send에 아무것도 안 입력하면 None이 전달된다.

>> coroutine started.
>> coroutine received : 100


StopIteration: 

In [13]:
# 잘못된 사용
cr2 = coroutine1()
cr2.send(100)
# TypeError: can't send non-None value to a just-started generator
# 이제 막 시작된 제네레이터는 값 전송이 안 된다.

TypeError: can't send non-None value to a just-started generator

state 정보 <br>
- GEN_CREATED : 처음 대기 상태
- GEN_RUNNING : 실행 상태
- GEN_SUSPENDED : yield 대기 상태 -> 정보 송수신 가능
- GEN_CLOSED : 실행 완료 상태

In [17]:
# coroutine2

# 수신 yield 송신

def coroutine2(x):
    print(f'>> coroutine started : {x}')

    y = yield x
   
    print(f'>> coroutine received : {y}')
    # closure: free variable의 상태를 저장한 것이다.
    z = yield x + y

    print(f'>> coroutine received : {z}')




In [19]:
from inspect import getgeneratorstate



cr3 = coroutine2(10)

print(next(cr3))
# yield x까지 실행되었고 y값을 받을 대기상태

print(getgeneratorstate(cr3)) # y = yield 수신 대기 중

print(cr3.send(100))
# y = yield부터 시작해서 main routine한테 100을 받는다.
# z값을 받기 위해 대기상태

print(getgeneratorstate(cr3)) # z = yield 수신 대기 중

print(cr3.send(200))
#  z = yield부터 시작해서 종료

print(getgeneratorstate(cr3))

>> coroutine started : 10
10
GEN_SUSPENDED
>> coroutine received : 100
110
GEN_SUSPENDED
>> coroutine received : 200


StopIteration: 

python 3.5 이상의 버전에서 <br>
- def -> async 
- yield -> await (StopIteration 자동처리)

대체할 수 있다. 완전 비동기할 때는 짝을 맞춰야 하지만
def를 그냥 사용하면 그냥 함수인지 아닌지 알기 힘들어서 사용한다.

In [1]:
# 코루틴3
# StopIteration 자동 처리 (3.5 이상 await)

# 중첩 코루틴 처리 (nested coroutine)

def generator1():
    for x in 'AB':
        yield x

    for y in range(1,4):
        yield y

t1 = generator1()

print(next(t1))


TypeError: 'async_generator' object is not an iterator

In [27]:
print(next(t1))
print(next(t1))
print(next(t1))
print(next(t1))
print(next(t1))



B
1
2
3


StopIteration: 

In [28]:
t2 = generator1()
print(list(t2)) # range가 list 안에서 처리되는 방식이다.



['A', 'B', 1, 2, 3]


yield from iterable_object <br>
iterable_obeject가 끝날 때까지 반환하겠다. <br>

In [32]:
def generator2():

    yield from 'AB'
    yield from range(1,4)

    

In [33]:
t3 = generator2()



In [34]:
print(next(t3))
print(next(t3))
print(next(t3))
print(next(t3))
print(next(t3))
print(next(t3))

A
B
1
2
3


StopIteration: 

close() 함수를 이용하면 generator를 깔끔하게 닫을 수 있다. <br>
아직 중간지점이라도 close를 사용하면 GeneratorExit 예외를 발생시키기 때문이다. <br>


In [3]:
def generator2():
    try:
        yield from 'AB'
        yield from range(1,4)
    except GeneratorExit:
        print("Closing coroutine!")

coro = generator2()
next(coro)
coro.close()


Closing coroutine!
